In [ ]:
###Detailed refernceing and methodology is written in the submitted paper methods section###
###If there are any questions about the code image analysis please feel free to contact me at my private email: yagmur2986@gmail.com###
###Further Supervision contacts for project specific questions: Sebastian Gilbert and Georgiana Neag###

import tifffile as tiff
import imageio.v3 as iio
import matplotlib.pyplot as plt
import skimage as ski
import napari
import time
import skan 
import skimage.io
import numpy as np
import pandas as pd 
from scipy import ndimage as ndi 
from skimage.exposure import rescale_intensity
from skimage.filters import meijering, threshold_otsu 
from skimage.transform import rescale 
from skimage.morphology import skeletonize_3d
from skimage import data, filters, morphology
from skimage.morphology import skeletonize_3d
from skan import Skeleton

In [ ]:
###DENDRITE FILE READ AND NORMALISATION###

#To visualise computational time taken for each step#
total_start = time.time()

#Import files as tiff for better readability and runtime#
#insert the relevent files and make sure they are in the corerect dimentions (x,y,z): there is a checkpoint below#
start = time.time()
print ("reading files...")
dendrite_original = tiff.imread('[Enter_File]') 
dendrite_probability = tiff.imread('[Enter_File]') #will be the forgound channel
print (f"files loaded. Time taken: {time.time() - start: .2f} seconds")

#This allows to check for the dimentions / shape of the files, ensure both are the same,
#if one has an additional parameter e.g. (66, 2, 4202, 1043) it means there are two channels, 
#need to therefore adjust.
print ("Dendrites original shape:", dendrite_original.shape)
print ("Dendrites probability shape:", dendrite_probability.shape)

#complete image normalisation (takes a high amount of computational power)
start = time.time()
print ("normalising files...")
image_normalisation = rescale_intensity(dendrite_probability, out_range=(0,1))
print(f'Intensity range: [{image_normalisation.min()} - {image_normalisation.max()}]')
print(f'Array type: {image_normalisation.dtype}')
print (f"Normalisation done. Time taken: {time.time() - start: .2f} seconds")

#Vesselness filtering through meijering#
#Note Meijering usually takes the longest for up to over 20minutes, please give it time#
start = time.time()
print ("meijering...")
dendrite_meijering = meijering(image_normalisation, sigmas=range(1,3,1), black_ridges=False)
print (f"Meijering done. Time taken: {time.time() - start: .2f} seconds")

np.save("[Enter_Save_Name]", dendrite_meijering)

In [ ]:
dendrite_meijering = np.load("[Enter_Name]") #Optional reload for later use
dendrite_original = tiff.imread('Data/Dendrites_YM4-first-batch-Hoechst-Phall555-63x-merged.ims - YM4-first-batch-Hoechst-Phall555-63x-merged.ims Resolution Level 1 - C=1.tif') 
dendrite_probability = tiff.imread('Data/YM4_Dendrite_Foreground.tif') #will be the forgound channel

In [ ]:
###DENDRITE THRESHOLDING, DENOISING AND CLEANING AND SKELETONISATION STEP###

#Thresholding: Two metjods# 
#Otsu's method not suitable for images with uneven illumination or non-uniform background#
#Li's method is, this version will explore otsu method# 
start = time.time()
print ("denoising, thresholding and cleaning...")
labels_binary = (dendrite_meijering >= threshold_otsu(dendrite_meijering)).astype(np.uint8)
cleaned = morphology.remove_small_objects(
    morphology.remove_small_holes(labels_binary, 2**3),
    20**3)
print (f"otsu threshold done. Time taken: {time.time() - start: .2f} seconds")

#Checkpoint view with 2D slide bar 
viewer = napari.Viewer()

In [ ]:
###RESCALE VOXELS ON BINARY FILE###

original_dendrite_voxel = ['Enter Voxels'] #Enter voxel in format e.g. 0.299, 0.241, 0.241
wanted_dendrite_voxel =  #Enter desired voxel for rescaling e.g. 0.241

scale_factors = [ x/wanted_dendrite_voxel for x in original_dendrite_voxel] 

print ("Rescaling...")
dendrite_otsu_rescale = rescale(
    labels_binary,
    scale=scale_factors,
    order=1,
    preserve_range=True,
    anti_aliasing =True, 
    channel_axis = None)
print (f"Rescaling done. Time taken: {time.time() - start: .2f} seconds")

In [ ]:
###CHECKPOINT###

#Check shape and datatype of both original otsu thresholded binary labels and the calculated rescale#
print ("Dendrite original otsu shape:", labels_binary.shape)
print ("Dendrite otsu rescale shape:", dendrite_otsu_rescale.shape)
print ("Dendrite original otsu datatype:", labels_binary.dtype)
print ("Dendrite otsu rescale datatype:", dendrite_otsu_rescale.dtype)

#Performed standard thresholding on the rescale with: anything not 0 is 1 therfore, binary threshold was performed#
BINARY_rescaled_dendrite_otsu_rescale_uint = (dendrite_otsu_rescale != 0).astype(np.uint8)

#Confirm and validate results#
print ("BINARY THRESHOLDED Dendrite otsu rescale datatype:", BINARY_rescaled_dendrite_otsu_rescale_uint.dtype)
print ("BINARY THRESHOLDED Dendrite otsu rescale shape:", BINARY_rescaled_dendrite_otsu_rescale_uint.shape)

In [ ]:
###SAVE CHECKPOINT###

tiff.imwrite('[Enter_Save_Name]', BINARY_rescaled_dendrite_otsu_rescale_uint)
print ('file saved')

In [ ]:
###FILE OPEN AND CHECKPOINT###

dendrite_rescaled_binarised_open = tiff.imread('[Enter_File]')
print ("Dendrite otsu rescale shape:", dendrite_rescaled_binarised_open.shape)
print (dendrite_rescaled_binarised_open.dtype)

In [ ]:
###POST-RESCALING SKELETONISATION###

#Skeletonisation after rescaling# 
#Can get a C++ runtime error here, if this happens please re run previous cells and try again, if not, restart kernel#
start = time.time()
print ("skeletonising..")
dendrite_skeleton_rescale = skeletonize_3d(dendrite_rescaled_binarised_open)
viewer.add_image(
    dendrite_skeleton_rescale,
    name = "Skeleton",
    contrast_limits = (0,1),
    colormap='green',
    blending='additive'
)
print (f"skeletonisation done. Time taken: {time.time() - start: .2f} seconds")

In [ ]:
###POST-RESCALINE DATAFRAME###
table = skan.summarize(Skeleton(skeleton_image = dendrite_skeleton_rescale))
table.drop(table[table['branch-type'] == 0].index, axis=0, inplace=True)
table.head

In [ ]:
###VECTORS###

#Note: This will generate branch types 1,2 and 3. These are as follows:
# Branch type 1: Dendrites that have only one attachment and have a dead end. 
# Branch type 2: Dendrites that have multiple attachmetns and connections.
# Branch type 3: Dendrites that form loops.

points_src = table[['image-coord-src-0', 'image-coord-src-1', 'image-coord-src-2']].values
points_dst = table[['image-coord-dst-0', 'image-coord-dst-1', 'image-coord-dst-2']].values 

directions = points_dst - points_src 

vectors = np.concatenate((points_src[np.newaxis], directions[np.newaxis]), axis=0)
vectors = np.swapaxes(vectors, 0,1)
vectors.shape

In [ ]:
###NAPARI VISUALISATION###

#Run Napari to ensure it works 
viewer = napari.Viewer()
viewer.dims.ndisplay = 3
viewer.add_image(dendrite_original, name='Dendrite Original', colormap='grey') # 1. original image#
viewer.add_image(dendrite_probability, name='Dendrite Probability', colormap='viridis') # 2. probability image#
viewer.add_image(dendrite_meijering, name="Vesselness filter", contrast_limits=(0, 1)); # 3. meijering vesselness filter#
viewer.add_labels(labels_binary, name="Otsu Threshold") # 4. pre-rescaled otsu threshold binarised#
viewer.add_labels(dendrite_rescaled_binarised_open, name="Otsu Threshold RESCALED") # 5. pre-rescaled otsu threshold binarised#
viewer.add_image(dendrite_skeleton, name = "Skeleton", contrast_limits = (0,1), colormap='cyan', blending='additive') # 6. pre-rescaled skeleton#
viewer.add_image(dendrite_skeleton_rescale, name = "Skeleton RESCALE", contrast_limits = (0,1), colormap='green', blending='additive') #7. post-rescaled skeleton#
viewer.add_vectors(vectors, name="Branch vectors", edge_width=0.6,
                   features=pd.DataFrame({'branch-type': table['branch-type'].values.astype(float)}), 
                   edge_color='branch-type', vector_style='arrow',
                  ) # 8. Vectors (type 1,2,3)#

In [ ]:
# File saves to use in the analysis script # 

#Save datafame as csv#
table.to_csv("[Enter_Save_Name]" , index = False)

#save img as tiff#
tiff.imwrite('[Enter_Save_Name]', labels_binary)
tiff.imwrite('[Enter_Save_Name]', dendrite_rescaled_binarised_open)

#save array as npy#
np.save("[Enter_Save_Name]", dendrite_meijering)
np.save("[Enter_Save_Name]", dendrite_skeleton_rescale)
np.save("[Enter_Save_Name]", vectors)